In [2]:
import cv2
import os
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder

In [11]:
tf.keras.backend.clear_session()

In [3]:
landmarks = np.load('landmarks.npz')
labels = np.load('labels.npz')
landmarks = landmarks['arr']
labels = labels['arr']

In [4]:
landmarks = landmarks.reshape(372, 42, 3)

In [5]:
landmarks.shape,labels.shape

((372, 42, 3), (372,))

In [7]:
from sklearn.model_selection import train_test_split
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(landmarks, labels, test_size=0.2, random_state=42)

In [8]:
decoded_labels = label_encoder.inverse_transform(labels)

In [9]:
labels[370],decoded_labels[370]

(7, 7)

In [10]:
import tensorflow as tf

# Define the CNN model
def create_base_cnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu')
    ])
    return model

# Define input shape and number of classes
input_shape = (42, 3)  # Example input shape for 1D data with 100 timesteps and 1 feature
num_classes = 8  # Example number of classes

# Create the model
base_model = create_base_cnn_model(input_shape)
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

model = tf.keras.Sequential([base_model, output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 64)                59904     
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 60424 (236.03 KB)
Trainable params: 60424 (236.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50


10/10 [==============================] - 3s 7ms/step - loss: 2.0805 - accuracy: 0.1616
Epoch 2/50
10/10 [==============================] - 0s 6ms/step - loss: 2.0583 - accuracy: 0.1852
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 2.0497 - accuracy: 0.2189
Epoch 4/50
10/10 [==============================] - 0s 5ms/step - loss: 2.0382 - accuracy: 0.2290
Epoch 5/50
10/10 [==============================] - 0s 5ms/step - loss: 2.0069 - accuracy: 0.2525
Epoch 6/50
10/10 [==============================] - 0s 6ms/step - loss: 1.9663 - accuracy: 0.2727
Epoch 7/50
10/10 [==============================] - 0s 5ms/step - loss: 1.9318 - accuracy: 0.2727
Epoch 8/50
10/10 [==============================] - 0s 6ms/step - loss: 1.8665 - accuracy: 0.3367
Epoch 9/50
10/10 [==============================] - 0s 5ms/step - loss: 1.7994 - accuracy: 0.3333
Epoch 10/50
10/10 [==============================] - 0s 6ms/step - loss: 1.7294 - accuracy: 0.3838
Epoch 11/50
10/10

In [12]:
model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.5011 - accuracy: 0.8133


[0.5010682940483093, 0.8133333325386047]

In [13]:
base_model.save('base.keras')